In [0]:
import pandas as pd
#import pyspark.pandas as pd


In [0]:
df= dbutils.fs.ls("/mnt/something/")
display(df)

In [0]:
white_wine =  pd.read_csv("/dbfs/mnt/something/winequality-white.csv",sep=";")
red_wine = pd.read_csv("/dbfs/mnt/something/winequality-red.csv",sep=";")

red_wine['is_red']=1
white_wine['is_red']=0

display(red_wine)
display(white_wine)


In [0]:
# def format_column_names(df):
#     """Rename columns """
#     renamed_df = df
#     for col in df.columns:                
#         renamed_df = renamed_df.withColumnRenamed(col, col.replace(" ", "_").lower())                                          
#     return renamed_df

In [0]:
# concat the two datasets
concat_df = pd.concat([white_wine, red_wine], ignore_index=True)
display(concat_df)

In [0]:
# renamed_df = format_column_names(concat_df) 
renamed_df = concat_df.rename(columns=lambda x: x.replace(" ", "_").lower())
display(renamed_df)

In [0]:
display(renamed_df.describe)

In [0]:
renamed_df.corr

In [0]:
display(renamed_df.head())

In [0]:
renamed_df.dtypes

In [0]:
renamed_df.describe()

In [0]:
renamed_df.corr()

In [0]:

display(renamed_df)

Databricks visualization. Run in Databricks to view.

In [0]:
from sklearn.model_selection import train_test_split

X = renamed_df.drop("quality",axis=1)  # Specify axis=1 to drop a column
y = renamed_df["quality"]

# split out the training data 
X_train, X_remaining, y_train, y_remaining = train_test_split(X, y, test_size=0.2, random_state=42)


In [0]:
# split the remaining data equally into validation and test
X_validation, X_test, y_validation, y_test = train_test_split(X_remaining, y_remaining, test_size=0.5, random_state=123)

### Build baseline model

In [0]:
import mlflow
import mlflow.pyfunc
import mlflow.sklearn
import numpy as np

import sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from mlflow.models.signature import infer_signature
from mlflow.utils.environment import _mlflow_conda_env
import cloudpickle
import time

In [0]:
class SklearnModel(mlflow.pyfunc.PythonModel):
    def __init__(self, model): #Constructor, takes in the model 
        self.model = model

    def predict(self, context, model_input): # implementation of pyfunc.PythonModel      
        return self.model.predict_proba(model_input)[:,1]
    

with mlflow.start_run(run_name="untuned radom forest"):
    n_estimators = 10
    model = RandomForestClassifier(n_estimators=n_estimators, random_state=np.random.RandomState(123))
    model.fit(X_train, y_train)

    # predict proba returns [prob_negative, prob_positive], so slice the output with [:,1]
    predictions_test = model.predict_proba(X_test)[:,1]
    auc_Score = roc_auc_score(y_test, predictions_test)
    mlflow.log_param("n_estimators",n_estimators)

    # use the area under the ROC curve as a metric
    mlflow.log_metric("auc", auc_Score)
    wrappedModel = SklearnModel(model)

    # Log the model with a signature that defines the schema of the model's inputs and outputs.
    # when the model is deployed in production, the model will be loaded with this signature

    signature = infer_signature(X_train, wrappedModel.predict(None, X_train))

    # MLFLow contains utilities to create a conda environment used to serve models. The necessary dependencies are added to a conda.yaml file which is logged along with the model.
    conda_env = mlflow_conda_env(
        additional
    )







